# This is a notebook for querying the WOD instance for the 1.0.0 Beacon release.

- You can run each cell individually by pressing "shift + enter".
- For more information, questions, bugs, please contact us on Slack:
  - https://join.slack.com/t/beacontechnic-wwa5548/shared_invite/zt-2dp1vv56r-tj_KFac0sAKNuAgUKPPDRg.


#### In order to get access to the Beacon endpoint, you need to fill in your unique personal token between the " " in the cell below.


In [1]:
Token = ""

#### Install the following packages, if you have not already installed them in your environment:

- pip install requests
- pip install xarray
- pip install ipywidgets
- pip install cartopy
- pip install h5netcdf
- pip install netcdf4
- pip install scipy
- pip install packaging


#### Import the required packages


In [2]:
import requests
import json
import xarray as xr
import datetime
import pandas as pd
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

#### Retrieve the available columns from the WOD endpoint

Swagger page here: https://beacon-wod.maris.nl/swagger/


In [3]:
responseinfo = requests.get("https://beacon-wod.maris.nl/api/query/available-columns",
                            headers={"Authorization": f"Bearer {Token}"})

params = responseinfo.json()

#### Below you can search through the available columns by entering text between the brackets of search_columns(" ").


In [4]:
def search_columns(search_term):
    search_term = search_term.lower()
    matches = [col for col in params if search_term in col.lower()]

    if matches:
        print("Matching columns:")
        for match in matches:
            print(match)
    else:
        print("No matching columns found.")


search_columns("temp")  # Enter your search term here

Matching columns:
Air_Temperature
Air_Temperature._FillValue
Air_Temperature.coordinates
Air_Temperature.grid_mapping
Air_Temperature.long_name
Air_Temperature.units
Air_Temperature_WODflag
Air_Temperature_WODflag.flag_definitions
Air_Temperature_origflag
Air_Temperature_origflag.flag_definitions
Air_Temperature_sigfigs
Cal_Temp
Cal_Temp._FillValue
Cal_Temp.long_name
Cal_Temp.units
Dry_Bulb_Temp
Dry_Bulb_Temp._FillValue
Dry_Bulb_Temp.long_name
Dry_Bulb_Temp.units
Ref_Surf_Temp
Ref_Surf_Temp._FillValue
Ref_Surf_Temp.comment
Ref_Surf_Temp.units
Temp_Corr
Temp_Corr._FillValue
Temp_Corr.long_name
Temp_Corr.units
Temperature
Temperature._FillValue
Temperature.ancillary_variables
Temperature.coordinates
Temperature.grid_mapping
Temperature.long_name
Temperature.standard_name
Temperature.units
Temperature_Access_no
Temperature_Access_no._FillValue
Temperature_Access_no.comment
Temperature_Access_no.long_name
Temperature_Access_no.units
Temperature_Adjustment
Temperature_Adjustment._FillValue


#### You can define here your input parameters


In [5]:
parameter = "Temperature"  # column name
mindate = "2010-01-01"  # yyyy-mm-dd
maxdate = "2010-03-01"  # yyyy-mm-dd
minlon = -180
maxlon = 180
minlat = -90
maxlat = 90
mindepth = 0
maxdepth = 10

#### This will create the query body based on your input parameters, you can add other "query_parameters" and "filters" to suit your needs.

- For more query examples and explanations, you can take a look at https://github.com/maris-development/beacon-blue-cloud.


In [6]:
def query(parameter, mindate, maxdate, minlon, maxlon, minlat, maxlat, mindepth, maxdepth):
    body = {
        "query_parameters": [
            {
                "column_name": parameter,
                "alias": parameter,
                "skip_fill_values": True
            },
            {
                "column_name": f"{parameter}.units",
                "alias": "Unit"
            },
            {
                "column_name": "cf_datetime",
                "alias": "datetime"
            },
            {
                "column_name": "z",
                "alias": "DEPTH"
            },
            {
                "column_name": "lon",
                "alias": "LONGITUDE"
            },
            {
                "column_name": "lat",
                "alias": "LATITUDE"
            },
            {
                "column_name": "dataset",
                "alias": "DATASET",
                "optional": True
            },
            {
                "column_name": "WOD_cruise_identifier",
                "alias": "cruise-identifier",
                "optional": True
            },
            {
                "column_name": "wod_unique_cast",
                "alias": "cast",
                "optional": True
            },
            {
                "column_name": "WMO_ID",
                "alias": "WMO_ID",
                "optional": True
            },
            {
                "column_name": "country",
                "alias": "country",
                "optional": True
            }
        ],
        "filters": [
            {
                "for_query_parameter": "datetime",
                "min": f"{mindate}T00:00:00",
                "max": f"{maxdate}T00:00:00",
                "cast": "timestamp"
            },
            {
                "for_query_parameter": "DEPTH",
                "min": mindepth,
                "max": maxdepth
            },
            {
                "for_query_parameter": "LONGITUDE",
                "min": minlon,
                "max": maxlon
            },
            {
                "for_query_parameter": "LATITUDE",
                "min": minlat,
                "max": maxlat
            }
        ],
        "output": {
            "format": "ipc"
        }}
    return body


query_body = query(parameter, mindate, maxdate, minlon,
                   maxlon, minlat, maxlat, mindepth, maxdepth)

#### This is the post request that is sent to Beacon with the above specified body.


In [7]:
response = requests.post("https://beacon-wod.maris.nl/api/query", json.dumps(query_body), headers={
    "Authorization": f"Bearer {Token}",
    "Content-type": "application/json"
})

if response.status_code == 204:
    print("No data has been found for your query, please update your input fields above and run the notebook again.")
elif response.status_code != 200:
    print(response.text)

In [8]:
regionname = f"[{minlat},{minlon}],[{maxlat},{maxlon}]"

if not os.path.exists("./Beacon_V1.0.0_Output"):
    os.makedirs("Beacon_V1.0.0_Output")

open(
    f"./Beacon_V1.0.0_Output/WOD_{parameter}_{regionname}_{mindate}-{maxdate}_[{mindepth}-{maxdepth}m].arrow", "wb").write(response.content)
df = pd.read_feather(
    f"./Beacon_V1.0.0_Output/WOD_{parameter}_{regionname}_{mindate}-{maxdate}_[{mindepth}-{maxdepth}m].arrow")

# df = xr.open_dataset(f"./Beacon_V1.0.0_Output/WOD_{parameter}_{regionname}_{mindate}-{maxdate}_[{mindepth}-{maxdepth}m].nc").to_dataframe() For Netcdf

df = df.assign(datetime=pd.to_datetime(
    df["datetime"])).set_index("datetime").sort_index()
df

,Temperature,Unit,DEPTH,LONGITUDE,LATITUDE,DATASET,cruise-identifier,cast,WMO_ID,country,dataset_id
datetime,,,,,,,,,,,
2010-01-01,26.650000,degree_C,5.0,-22.990000,-0.01,moored buoy,FR014546,12327917,2.115093e+09,FRANCE,11382433
2010-01-01,27.969999,degree_C,1.0,-94.980003,4.97,moored buoy,US015956,12327891,2.115095e+09,UNITED STATES,11382375
2010-01-01,27.650000,degree_C,1.0,-9.870000,0.01,moored buoy,FR013665,12327916,2.115077e+09,FRANCE,11382436
2010-01-01,29.700001,degree_C,1.5,147.020004,1.99,moored buoy,JP030524,12327867,2.115114e+09,JAPAN,11382395
2010-01-01,30.280001,degree_C,1.0,-170.020004,-5.00,moored buoy,US015935,12327898,2.115114e+09,UNITED STATES,11382414
...,...,...,...,...,...,...,...,...,...,...,...
2010-03-01,29.049999,degree_C,1.5,147.029999,4.96,moored buoy,JP020811,12332286,2.115114e+09,JAPAN,11389957
2010-03-01,29.820000,degree_C,1.0,164.990005,-2.00,moored buoy,US015901,12332280,2.115114e+09,UNITED STATES,11389955
2010-03-01,29.900000,degree_C,10.0,80.500000,-1.53,moored buoy,99005823,12327388,2.115085e+09,UNKNOWN,11390010


#### Optional convert output to NetCDF and store


In [9]:
# Convert index to datetime dtype as older versions of xarray do not support datetime64[ns]
df.index = df.index.astype('datetime64[ns]')

ds = df.to_xarray()


ds.to_netcdf(
    f'./Beacon_V1.0.0_Output/WOD_{parameter}_{regionname}_{mindate}-{maxdate}_[{mindepth}-{maxdepth}m].nc')

#### Optionally apply a filter on your parameter for quick removal of outliers. Note that this can also be achieved within your Beacon request by applying a filter on your parameter.


In [10]:
highbound = 40
lowbound = -2
df = df.loc[(df[f"{parameter}"] < highbound) & (df[f"{parameter}"] > lowbound)]
df

,Temperature,Unit,DEPTH,LONGITUDE,LATITUDE,DATASET,cruise-identifier,cast,WMO_ID,country,dataset_id
datetime,,,,,,,,,,,
2010-01-01,26.650000,degree_C,5.0,-22.990000,-0.01,moored buoy,FR014546,12327917,2.115093e+09,FRANCE,11382433
2010-01-01,27.969999,degree_C,1.0,-94.980003,4.97,moored buoy,US015956,12327891,2.115095e+09,UNITED STATES,11382375
2010-01-01,27.650000,degree_C,1.0,-9.870000,0.01,moored buoy,FR013665,12327916,2.115077e+09,FRANCE,11382436
2010-01-01,29.700001,degree_C,1.5,147.020004,1.99,moored buoy,JP030524,12327867,2.115114e+09,JAPAN,11382395
2010-01-01,30.280001,degree_C,1.0,-170.020004,-5.00,moored buoy,US015935,12327898,2.115114e+09,UNITED STATES,11382414
...,...,...,...,...,...,...,...,...,...,...,...
2010-03-01,29.049999,degree_C,1.5,147.029999,4.96,moored buoy,JP020811,12332286,2.115114e+09,JAPAN,11389957
2010-03-01,29.820000,degree_C,1.0,164.990005,-2.00,moored buoy,US015901,12332280,2.115114e+09,UNITED STATES,11389955
2010-03-01,29.900000,degree_C,10.0,80.500000,-1.53,moored buoy,99005823,12327388,2.115085e+09,UNKNOWN,11390010


#### Plotting of results.


In [11]:
fig = plt.figure(figsize=(23, 18))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())  # (west, east, south, north)

ax.coastlines(resolution="10m")
ax.gridlines(draw_labels=True)

sc = ax.scatter(df["LONGITUDE"], df["LATITUDE"], c=df[parameter], cmap="viridis", s=15, transform=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.RIVERS)     
ax.add_feature(cfeature.BORDERS)   
ax.add_feature(cfeature.LAKES, alpha=0.1) 

cbar = plt.colorbar(sc, ax=ax, orientation="vertical", shrink=0.6, label="Value Set 1")
cbar.set_label(f"{parameter} [{df["Unit"].iloc[0]}]")

plt.title(f"{parameter} {regionname} {mindate}-{maxdate} [{mindepth}-{maxdepth}m]")

plt.show()

SyntaxError: f-string: unmatched '[' (1847372245.py, line 18)